In [1]:
from cdx.cdx_helper import cdx_scan, CRAWL_CDX

total_bytes = 0
i = 0

for cdx in cdx_scan('https://www.bankofengland.co.uk/', cdx_service=CRAWL_CDX, limit=1000000):
    if cdx.statuscode == '200':
        total_bytes += int(cdx.length)
        i += 1
    if i%1000 == 0:
        print(total_bytes)
    #print(cdx.length, cdx.original, cdx.statuscode)

230757
230757
230757
230757
230757
230757
230757
230757
44727342
453709647
453709647
485070742
485070742
485070742
485070742
485070742
700816261
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
801649324
805651500
805651500
805651500
805651500
805651500
805651500
805651500
805651500
805651500
805651500
808831251
817914489
822997565
827550186
827550186
828456539
838244057
845179100
845270113
853511615
858360416
858816009
858816009
858816009
858834831
878538566
906614755
917748402
919317730
929730563
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
929759266
93095